### 저장된 모델 로드 & 활용
- 모델 파일 종류	
	* 가중치 및 절편 저장 파일	=> 동일한 구조의 모델 인스턴스 생성 필요
	* 모델 전체 저장 파일		=> 바로 로딩 후 사용 가능

[1] 모듈로딩 <hr>

In [1]:
# 모듈 로딩
import torch
import torch.nn as nn

In [2]:
## 모델 파일 관련
# 모델 파일 저장
import os

# 저장경로
SAVE_PATH= r'C:/Users/KDP-43/Desktop/딥러닝/0919/models/iris/BCF/'

# 저장 파일명
SAVE_MODEL = SAVE_PATH+'BiCFmodel_all.pth'


In [3]:
# 모듈 로딩
# - Model관련
import torch              
import torch.nn as nn              
import torch.nn.functional as F 
from torchinfo import summary 

from BCF_Models import IrisBCFModel


- 모델 로드할 때,
	* 해당 모델 클래스가 import 되어야 함
	* .py로 class 생성해서 import 하면 됨

In [7]:
class IrisBCFModel(nn.Module):
    
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(4, 10)
        self.hd_layer=nn.Linear(10, 5)
        self.out_layer=nn.Linear(5, 1)
        
    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        #- 입력층
        y=self.in_layer(input_data)   
        y=F.relu(y)                   
        
        # - 은닉층 : 10개의 숫자 값(>=0)
        y=self.hd_layer(y)            
        y=F.relu(y)                  
        
        #- 출력층 : 5개의 숫자 값 => 이진분류
        return F.sigmoid(self.out_layer(y))         


[2] 모델 로딩 - 모델 전체 파일 사용

In [8]:
if os.path.exits(SAVE_MODEL):
	irisModel = torch.load(SAVE_MODEL, weights_only=False)
else:
	print(f'{SAVE_MODEL} 파일이 존재하지 않습니다. 다시 확인하세요.')

In [9]:
summary(irisModel)

Layer (type:depth-idx)                   Param #
IrisBCFModel                             --
├─Linear: 1-1                            50
├─Linear: 1-2                            55
├─Linear: 1-3                            6
Total params: 111
Trainable params: 111
Non-trainable params: 0

[3] 예측 <hr>

In [11]:
# data = [ float(x) for x in input("SL, SW, PL, PW: ").split(",") ]
data = [1.33, 2.83, 5.32, 3.42]

In [12]:
dataTS = torch.FloatTensor(data).reshape(1, -1)
dataTS, dataTS.shape

(tensor([[1.3300, 2.8300, 5.3200, 3.4200]]), torch.Size([1, 4]))

In [13]:
# 새로운 데이터에 대한 예측 -> predict
irisModel.eval()

with torch.no_grad():
    
	# 추론/평가
    pre_val = irisModel(dataTS)

print("이진 분류 양성 확률:", pre_val)
print("이진 분류 음성 확률:", 1-pre_val)
print()
print(pre_val > 0.5)
print((pre_val > 0.5).item())

tensor([[0.5159]])
tensor([[True]])
True


- 이진분류 -> 양성값(True;1)일 확률 1개만 반환
- 여기서는 Setosa 라벨을 '1'으로 인코딩

In [16]:
result ='Setosa' if (pre_val > 0.5).item() else 'Other'
print(result)

Setosa
